In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from mc2.data_management import DATA_ROOT, CACHE_ROOT, load_data_into_pandas_df

In [ ]:
mat_lbl = "N49"
b, h, t = load_data_into_pandas_df(material=mat_lbl, number=1)
b.shape, h.shape, t.shape

In [ ]:
fig, axes = plt.subplots(5, 2, figsize=(9, 5), sharex=True, sharey="col")

choice = np.random.choice(len(b), 5)
for i, c in enumerate(choice):
    axes[i, 0].plot(b.to_numpy()[c, :].T)
    axes[i, 1].plot(h.to_numpy()[c, :].T, color='tab:orange')
for ax in axes.flatten():
    ax.grid(alpha=0.5)
axes[0, 0].set_title("B in mT")
axes[0, 1].set_title("H in A/m")
for ax in axes[-1, :]:
    ax.set_xlabel("Step")
fig.suptitle(mat_lbl)

In [ ]:
choice